In [17]:
import pandas as pd

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
stock_df = pd.read_csv('stock.csv',index_col=0)
stock_df

,Name,Symbol,marketcap,price (USD),country
Rank,,,,,
1,Apple,AAPL,3.068670e+12,195.1000,United States
2,Microsoft,MSFT,2.745270e+12,355.0800,United States
3,Alphabet,GOOG,1.544570e+12,122.7800,United States
4,Amazon,AMZN,1.388850e+12,135.3600,United States
5,NVIDIA,NVDA,1.162800e+12,470.7700,United States
...,...,...,...,...,...
932,Quanergy Systems,QNGY,1.196165e+06,0.0740,United States
933,Triterras,TRIRF,3.826200e+04,0.0005,Singapore
934,Pareteum Corporation,TEUM,1.427000e+04,0.0000,United States


In [19]:
stock_df = stock_df.loc[stock_df['country'] == 'United States']
stock_df

,Name,Symbol,marketcap,price (USD),country
Rank,,,,,
1,Apple,AAPL,3.068670e+12,195.1000,United States
2,Microsoft,MSFT,2.745270e+12,355.0800,United States
3,Alphabet,GOOG,1.544570e+12,122.7800,United States
4,Amazon,AMZN,1.388850e+12,135.3600,United States
5,NVIDIA,NVDA,1.162800e+12,470.7700,United States
...,...,...,...,...,...
930,Grom Social Enterprises,GROM,3.130414e+06,0.3481,United States
932,Quanergy Systems,QNGY,1.196165e+06,0.0740,United States
934,Pareteum Corporation,TEUM,1.427000e+04,0.0000,United States


In [20]:
i = 0
for index, row in stock_df.iterrows():
    print(row['Name'])
    i += 1
    if i == 10:
        break

Apple
Microsoft
Alphabet
Amazon
NVIDIA
Tesla
Meta Platforms
Broadcom
Oracle
Adobe


### 크롤링

In [25]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
options.add_experimental_option('excludeSwitches', ['enable-logging']) # Usb Error ignore
options.add_argument("no-sandbox") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('user-agent='+ user_agent)

driver = webdriver.Chrome(options = options)
driver.implicitly_wait(15) # 묵시적 대기, 활성화를 최대 15초까지 기다린다.
# 페이지 가져오기(이동)
driver.get("https://www.dnb.com/business-directory/company-profiles.apple_inc.ec7f550b3a97b94d919d837672573959.html")  

# 더 많은 요소 보기 클릭(See All Industries)
a_tag = driver.find_element(By.CSS_SELECTOR, '#page > div.content.content-body > div.main_content > div.content-par.parsys > div.company_profile_overview.basecomp.RootBaseComponent.section > div > div > div > div.company_profile_overview_body > div > div.col-md-6.company-profile-overview-starting-margin > div:nth-child(5) > span > span.overview-industries-view-more > a')
a_tag.click()

ind_list = driver.find_element(By.CSS_SELECTOR, '#page > div.content.content-body > div.main_content > div.content-par.parsys > div.company_profile_overview.basecomp.RootBaseComponent.section > div > div > div > div.company_profile_overview_body > div > div.col-md-6.company-profile-overview-starting-margin > div:nth-child(5) > span > span:nth-child(2)')
ind_more_list = driver.find_element(By.CSS_SELECTOR, '#page > div.content.content-body > div.main_content > div.content-par.parsys > div.company_profile_overview.basecomp.RootBaseComponent.section > div > div > div > div.company_profile_overview_body > div > div.col-md-6.company-profile-overview-starting-margin > div:nth-child(5) > span > span.overview-view-more-industries-list')
print(ind_list.text + ind_more_list.text)

    

Communications Equipment Manufacturing ,  Computer and Peripheral Equipment Manufacturing ,  Audio and Video Equipment Manufacturing ,  Software Publishers ,  Mobile communication equipment,  Personal computers (microcomputers) ,  Computer terminals, monitors and components ,  Printers, computer ,  Sound reproducing equipment ,  Operating systems computer software Application computer software


In [31]:
# 전체 크롤링 코드
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from collections import defaultdict

options = Options()
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
options.add_experimental_option('excludeSwitches', ['enable-logging']) # Usb Error ignore
options.add_argument("no-sandbox") 
options.add_argument('headless')
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('user-agent='+ user_agent)
# options.add_argument('--proxy-server=socks5://127.0.0.1:9050')

driver = webdriver.Chrome(options = options)
driver.implicitly_wait(15) # 묵시적 대기, 활성화를 최대 15초까지 기다린다.
# 페이지 가져오기(이동)
driver.get("https://www.dnb.com/business-directory/company-profiles.apple_inc.ec7f550b3a97b94d919d837672573959.html")  

company = dict()
i = 0
for index, row in stock_df.iterrows():
    print(row['Name'], (len(stock_df)-i), "번 남았습니다.")
    i += 1
    if i == 5:
        break
    time.sleep(random.uniform(7,10)) # 서버 시도횟수
    # driver.implicitly_wait(15)
    driver.get(f"https://www.dnb.com/site-search-results.html#AllSearch={row['Name']}&CompanyProfilesPageNumber=1&CompanyProfilesSearch={row['Name']}&ContactProfilesPageNumber=1&DAndBMarketplacePageNumber=1&DAndBMarketplaceSearch=Apple&IndustryPageNumber=1&SiteContentPageNumber=1&SiteContentSearch={row['Name']}&tab=All") 
    
    try:
        ind_a_tag = driver.find_element(By.CSS_SELECTOR, '#page > div.content-ipar-cta.iparsys.parsys > div.SinglePageSearch.basecomp.RootBaseComponent.section > div > div > div.z_b105466b6478d8e0_tabContentsComponentWrapper.H7jcl87pRUHVsMGuetr_.z_b105466b6478d8e0_isActiveTab-true.bWQWIeQRobUvNlpeEeIt > div.z_5924c29f368a5561_topResultContainer.PkfTMnlfVSI87OftGJxB > div:nth-child(1) > div.z_b105466b6478d8e0_colorAccentWrapper.EmpMRt49dLK7BSfwaHwO > div.z_b105466b6478d8e0_searchResultsContainer.uBdFi1TMILYzved_X3wi > div.z_20f9dcf9d844d2bf_table-container.jzH4Iq8a0fLxtRaKzruO > table > tbody > tr:nth-child(1) > td.z_20f9dcf9d844d2bf_td-div.DUnLcYf0srtWETNfDOhS > a.z_20f9dcf9d844d2bf_tableCompanyNameLink.KzFf0D5CcLuHHQ6q6oQm')
        ind_a_tag.click()
    except:
        continue

    # 더 많은 요소 보기 클릭(See All Industries)
    try:
        a_tag = driver.find_element(By.CSS_SELECTOR, '#page > div.content.content-body > div.main_content > div.content-par.parsys > div.company_profile_overview.basecomp.RootBaseComponent.section > div > div > div > div.company_profile_overview_body > div > div.col-md-6.company-profile-overview-starting-margin > div:nth-child(5) > span > span.overview-industries-view-more > a')
        a_tag.click()

        ind_list = driver.find_element(By.CSS_SELECTOR, '#page > div.content.content-body > div.main_content > div.content-par.parsys > div.company_profile_overview.basecomp.RootBaseComponent.section > div > div > div > div.company_profile_overview_body > div > div.col-md-6.company-profile-overview-starting-margin > div:nth-child(5) > span > span:nth-child(2)')
        ind_more_list = driver.find_element(By.CSS_SELECTOR, '#page > div.content.content-body > div.main_content > div.content-par.parsys > div.company_profile_overview.basecomp.RootBaseComponent.section > div > div > div > div.company_profile_overview_body > div > div.col-md-6.company-profile-overview-starting-margin > div:nth-child(5) > span > span.overview-view-more-industries-list')
        ind_total_list = ind_list.text + ind_more_list.text

        company[row['Name']] = ind_total_list
    except:
        ind_list = driver.find_element(By.CSS_SELECTOR, '#page > div.content.content-body > div.main_content > div.content-par.parsys > div.company_profile_overview.basecomp.RootBaseComponent.section > div > div > div > div.company_profile_overview_body > div > div.col-md-6.company-profile-overview-starting-margin > div:nth-child(5) > span > span:nth-child(2)')
        # ind_more_list = driver.find_element(By.CSS_SELECTOR, '#page > div.content.content-body > div.main_content > div.content-par.parsys > div.company_profile_overview.basecomp.RootBaseComponent.section > div > div > div > div.company_profile_overview_body > div > div.col-md-6.company-profile-overview-starting-margin > div:nth-child(5) > span > span.overview-view-more-industries-list')
        # ind_total_list = ind_list.text + ind_more_list.text

        company[row['Name']] = ind_list.text

print(company)

# 딕셔너리를 데이터프레임으로 변환
df = pd.DataFrame(company.items(), columns=['Company', 'Service'])

# 데이터프레임을 엑셀 파일로 저장 (파일명: 'company_services.xlsx')
df.to_excel('company_services.xlsx', index=False)


Apple 525 번 남았습니다.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#page > div.content.content-body > div.main_content > div.content-par.parsys > div.company_profile_overview.basecomp.RootBaseComponent.section > div > div > div > div.company_profile_overview_body > div > div.col-md-6.company-profile-overview-starting-margin > div:nth-child(5) > span > span:nth-child(2)"}
  (Session info: headless chrome=115.0.5790.102); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104d9ff48 chromedriver + 4226888
1   chromedriver                        0x0000000104d984f4 chromedriver + 4195572
2   chromedriver                        0x00000001049dcd68 chromedriver + 281960
3   chromedriver                        0x0000000104a17fc8 chromedriver + 524232
4   chromedriver                        0x00000

In [12]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
options.add_experimental_option('excludeSwitches', ['enable-logging']) # Usb Error ignore
options.add_argument("no-sandbox") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('user-agent='+ user_agent)

driver = webdriver.Chrome(options = options)
driver.implicitly_wait(15) # 묵시적 대기, 활성화를 최대 15초까지 기다린다.
# 페이지 가져오기(이동)
driver.get("https://www.dnb.com/site-search-results.html#AllSearch=Apple&CompanyProfilesPageNumber=1&CompanyProfilesSearch=Apple&ContactProfilesPageNumber=1&DAndBMarketplacePageNumber=1&DAndBMarketplaceSearch=Apple&IndustryPageNumber=1&SiteContentPageNumber=1&SiteContentSearch=Apple&tab=All")  

ind_a_tag = driver.find_element(By.CSS_SELECTOR, '#page > div.content-ipar-cta.iparsys.parsys > div.SinglePageSearch.basecomp.RootBaseComponent.section > div > div > div.z_b105466b6478d8e0_tabContentsComponentWrapper.H7jcl87pRUHVsMGuetr_.z_b105466b6478d8e0_isActiveTab-true.bWQWIeQRobUvNlpeEeIt > div.z_5924c29f368a5561_topResultContainer.PkfTMnlfVSI87OftGJxB > div:nth-child(1) > div.z_b105466b6478d8e0_colorAccentWrapper.EmpMRt49dLK7BSfwaHwO > div.z_b105466b6478d8e0_searchResultsContainer.uBdFi1TMILYzved_X3wi > div.z_20f9dcf9d844d2bf_table-container.jzH4Iq8a0fLxtRaKzruO > table > tbody > tr:nth-child(1) > td.z_20f9dcf9d844d2bf_td-div.DUnLcYf0srtWETNfDOhS > a.z_20f9dcf9d844d2bf_tableCompanyNameLink.KzFf0D5CcLuHHQ6q6oQm')
print(ind_a_tag)
print(ind_a_tag.text)
ind_a_tag.click()

<selenium.webdriver.remote.webelement.WebElement (session="c97420c33389e425bf73d826e03b9f37", element="5E17430C546393859CF1AFB2E9B7B96B_element_23")>
Apple Inc.
